# Packages

In [1]:
import numpy as np
import pandas as pd
from astropy import units as u
from astropy import constants as c

# Ancillary functions

In [2]:
def get_atomic_gas_surface_density(W_21cm, W_21cm_err):
    """
    Compute atomic gas surface density following Walter+08
    
    Input:
    - W_21cm  : 21-cm line integrated intensity [K km/s]
    
    Output: 
    - Sd_atom : atomic gas surface density [Msun/pc^2]
    """

    # compute atomic gas surface density
    Sd_atom = 1.97e-2 * W_21cm * u.Msun/u.pc**2  # (Walter+08)
    Sd_atom_err = 1.97e-2 * W_21cm_err * u.Msun/u.pc**2  # (Walter+08)
    
    # convert to common units
    Sd_atom_value = Sd_atom.value
    Sd_atom_err_value = Sd_atom_err.value
    
    return Sd_atom_value, Sd_atom_err_value

# dense gas 
def get_dense_gas_surface_density(W_hcn, W_hcn_err, aHCN=10):
    """
    Compute dense gas surface density, e.g. like Gao&Solomon04 or others, but
    allowing more general, i.e. varying, HCN-to-dense gas conversion factor
    
    Input:
    - W_hcn  : HCN(1-0) line integrated intensity [K km/s]
    - aHCN   : HCN-to-dense gas mass conversion factor [Msun/pc^2/(K km/s)]
    
    Output: 
    - Sd_dense : dense gas surface density [Msun/pc^2]
    """

    # compute dense gas surface density
    Sd_dense = aHCN * W_hcn * u.Msun/u.pc**2 
    Sd_dense_err = aHCN * W_hcn_err * u.Msun/u.pc**2
    
    # convert to common units
    Sd_dense_value = Sd_dense.value
    Sd_dense_err_value = Sd_dense_err.value
    
    return Sd_dense_value, Sd_dense_err_value

# dynamical equilibrium pressure
def get_ism_pressure(W_co21, W_co21_err, W_21cm, W_21cm_err, Sd_star, Sd_star_err, l_star, 
                     sigma_gas_z=15, aCO=4.35, R21=0.65):
    """
    Compute dynamical equilibirium pressure following Sun+20
    
    Input:
    - W_co21  : CO(2-1) integrated intensity [K km/s]
    - W_21cm  : 21-cm line integrated intensity [K km/s]
    - Sd_star : stellar mass surface density [Msun/pc2]
    - l_star  : stellar disc scale length [kpc]
    - sigma_gas_z : vertical gas velocity dispersion [km/s]
    - aCO     : CO-H2 conversion factor [Msun/pc^2/(K km/s)]
    - R21     : CO(2-1)/CO(1-0) line ratio
    
    Output: 
    - pressure [kB K cm-3]
    """
    
    # assign units to input values
    Sd_star *= u.Msun/u.pc**2
    Sd_star_err *= u.Msun/u.pc**2
    l_star *= u.kpc
    sigma_gas_z *= u.km/u.s 

    # compute atomic gas surface density
    Sd_atom = 1.97e-2 * W_21cm * u.Msun/u.pc**2  # (Walter+08)
    Sd_atom_err = 1.97e-2 * W_21cm_err * u.Msun/u.pc**2  # (Walter+08)

    # compute molecular gas surface density
    Sd_mol = aCO / R21 * W_co21 * u.Msun/u.pc**2
    Sd_mol_err = aCO / R21 * W_co21_err * u.Msun/u.pc**2

    # compute total gas surface density
    Sd_gas = Sd_atom + Sd_mol
    Sd_gas_err = np.sqrt(Sd_atom_err**2 + Sd_mol_err**2)

    # compute stellar mass volume density
    rho_star = Sd_star / (0.55 * l_star)
    rho_star_err = Sd_star_err / (0.55 * l_star)

    # remove zeros and negatives
    rho_star[rho_star <= 0] = np.nan
    rho_star_err[rho_star <= 0] = np.nan

    # compute ISM (dynamical equilibrium) pressure
    P_DE = np.pi*c.G/2 * Sd_gas**2 + Sd_gas * np.sqrt(2*c.G * rho_star) * sigma_gas_z
    P_DE_err = np.sqrt( ((np.pi*c.G*Sd_gas + np.sqrt(2*c.G*rho_star)*sigma_gas_z)*Sd_gas_err)**2 
                     + ((Sd_gas/np.sqrt(2*c.G*rho_star)*c.G*sigma_gas_z)*rho_star_err)**2 )

    # convert to common units
    P_DE_value = (P_DE / c.k_B).cgs.value
    P_DE_err_value = (P_DE_err / c.k_B).cgs.value
    
    return P_DE_value, P_DE_err_value

# CO(2-1)/CO(1-0) calibration from Schinnerer & Leroy+24 (Tab. 1)
def get_R21_from_SFR(Sd_sfr):
    """
    Takes (resolved) star formation rate surface density and returns
    (resolved) CO(2-1)/CO(1-0) (R_21) line ratio.
    Can take multi-dim. arrays.
    
    Input:
    - Sd_sfr : star formation rate surface density [Msun/yr/kpc2]
    
    Output:
    - R_21   : CO(2-1)/CO(1-0) line ratio [dimensionless]
    """

    # set negatives to 0
    sfr = np.copy(Sd_sfr)
    sfr[sfr<0] = 0
    
    # apply scaling relation from table 1
    R_21 = 0.65 * (sfr/1.8e-2)**0.125


    # apply limits (table 1)
    R_21[R_21<0.35] = 0.35
    R_21[R_21>1] = 1
    
    return R_21

# Setup

In [3]:
# directories
wd = '/Users/lneumann/Documents/'
data_dir = wd + 'Data/'

# almond galaxies
almond_galaxies = ['ngc0628','ngc1097','ngc1365','ngc1385','ngc1511','ngc1546','ngc1566','ngc1672','ngc1792','ngc2566',
                   'ngc2903','ngc2997','ngc3059','ngc3521','ngc3621','ngc4303','ngc4321','ngc4535','ngc4536','ngc4569',
                   'ngc4826','ngc5248','ngc5643','ngc6300','ngc7496']
almond_res_list = ['18.6as', '19.4as', '20.6as', '19.9as', '17.6as', '18.9as', '19.7as', '18.2as', '18.7as', '18.5as',
                   '18.3as', '20.4as', '16.7as', '21.1as', '18.9as', '20.2as', '19.6as', '22.8as', '21.5as', '19.2as',
                   '18.7as', '19.9as', '18.0as', '17.7as', '17.9as']

# empire galaxies
empire_galaxies = ['ngc0628', 'ngc2903', 'ngc3184', 'ngc3627', 'ngc4254', 'ngc4321', 'ngc5055', 'ngc5194', 'ngc6946']
empire_res_list = ['33.3as', '33.3as', '33.3as', '33.3as', '33.3as', '33.3as', '33.3as', '33.3as', '33.3as']

# conversion factors
alpha_hcn = 15  # [M_Sun/pc^2/(K km/s)] (Schinnerer & Leroy 2024)
alpha_co10_mw = 4.35  # [M_Sun/pc^2/(K km/s)] (Bolatto et al. 2013)

# PHANGS sample table
phangs_sample_table_path = wd + 'Data/PHANGS/tables/phangs_sample_table_v1p6.csv'
phangs_sample_table = pd.read_csv(phangs_sample_table_path, comment='#', skiprows=1)

# ALMOND

In [4]:
for glxy, res, in zip(almond_galaxies, almond_res_list):

    # pystructure file
    pystruct_dir = data_dir + 'ALMOND/pystruct/'
    pystruct_file = pystruct_dir + '%s_data_struct_%s_spb2.npy' % (glxy, res)
    struct = np.load(pystruct_file, allow_pickle = True).item()
    
    ######## get data from structure ########

    # load distance and inclination
    dist_Mpc = struct['dist_mpc']  # distance to the galaxy in Mpc
    incl = struct['incl_deg']  # inclination of the galaxy in degree
    i = np.deg2rad(incl)  # convert from deg to rad
    
    # # conversion angular to linear scale
    dist_pc = dist_Mpc * 1e6
    kpc2as = 180/np.pi * 3600 / dist_pc * 1e3
    as2kpc = 1/kpc2as
    
    # CO(2-1) from PHANGS-ALMA
    co21 = np.array(struct['INT_VAL_CO21'])
    co21_err = np.array(struct['INT_UC_CO21'])
    
    # HCN(1-0) from ALMOND
    hcn = np.array(struct['INT_VAL_HCN10'])
    hcn_err = np.array(struct['INT_UC_HCN10'])
    
    # SFR surface density from z0MGS (GALEX+WISE)
    Sd_sfr = np.array(struct['INT_VAL_SFR_Z0MGS'])
    Sd_sfr_err = np.array(struct['INT_UC_SFR_Z0MGS'])
    
    # HI 21cm line intensity, K km/s (several surveys)
    hi = np.array(struct['INT_VAL_HI_21CM'])
    hi_err = np.array(struct['INT_UC_HI_21CM'])
    
    # Stellar mass surface density, Msun/pc2 (Spitzer 3.6um; Querejeta+15)
    Sd_star = np.array(struct['INT_VAL_MSTAR']) * np.cos(i)
    Sd_star_err = np.array(struct['INT_UC_MSTAR']) * np.cos(i)
    struct['INT_VAL_SD_STAR'] = Sd_star
    struct['INT_UC_SD_STAR'] = Sd_star_err
       
    # get CO(2-1)/CO(1-0) line ratio (SFR calibration from Schinnerer & Leroy 24)
    R_21 = struct['INT_VAL_R21_SL24'] 

    # get conversion factors
    alpha_co21_sl24 = struct['INT_VAL_ALPHACO21_SL24']
    alpha_co10_sl24 = struct['INT_VAL_ALPHACO10_SL24']

    
    ######## compute physical quantities ########

    # compute atomic gas surface density
    Sd_atom, Sd_atom_err = get_atomic_gas_surface_density(W_21cm=hi, W_21cm_err=hi_err)
    struct['INT_VAL_SD_ATOM'] = Sd_atom * np.cos(i)
    struct['INT_UC_SD_ATOM'] = Sd_atom_err * np.cos(i)

    # convert to dense gas surface density
    Sd_dense, Sd_dense_err = get_dense_gas_surface_density(W_hcn=hcn, W_hcn_err=hcn_err, aHCN=alpha_hcn)
    struct['INT_VAL_SD_DENSE'] = Sd_dense * np.cos(i)
    struct['INT_UC_SD_DENSE'] = Sd_dense_err * np.cos(i)

    # compute CO(1-0) intensity from CO(2-1) using above line ratio
    co10 = co21 / R_21
    co10_err = co21_err / R_21
    struct['INT_VAL_CO10'] = co10
    struct['INT_UC_CO10'] = co10_err

    # CO(1-0) intensities for stacking
    R_21_cube = np.broadcast_to(R_21, np.shape(struct['SPEC_VAL_CO21'].T)).T  # broadcast to cube shape
    struct['SPEC_VAL_CO10'] = np.copy(struct['SPEC_VAL_CO21']) / R_21_cube
    R_21_cube_shuff = np.broadcast_to(R_21, np.shape(struct['SPEC_VAL_SHUFFCO21'].T)).T  # broadcast to cube shape
    struct['SPEC_VAL_SHUFFCO10'] = np.copy(struct['SPEC_VAL_SHUFFCO21']) / R_21_cube_shuff
    struct['SPEC_VCHAN0_SHUFFCO10'] = np.copy(struct['SPEC_VCHAN0_SHUFFCO21'])
    struct['SPEC-DELTAV_SHUFFCO10'] = np.copy(struct['SPEC-DELTAV_SHUFFCO21'])
    
    ######## varying alpha_CO ########
    
    # compute molecular gas surface density
    Sd_mol_sl24 = co10 * alpha_co10_sl24
    Sd_mol_err_sl24 = co10_err * alpha_co10_sl24
    struct['INT_VAL_SD_MOL_SL24'] = Sd_mol_sl24 * np.cos(i)
    struct['INT_UC_SD_MOL_SL24'] = Sd_mol_err_sl24 * np.cos(i)

    # get disc scale length
    l_star_as = float(phangs_sample_table['size_scalelength'][phangs_sample_table['name']==glxy].iloc[0])
    l_star_kpc = l_star_as * as2kpc  # [kpc]
    
    # compute kpc-scale ISM equilibrium pressure
    P_DE_sl24, P_DE_err_sl24 = get_ism_pressure(co21*np.cos(i), co21_err*np.cos(i), hi*np.cos(i), hi_err*np.cos(i), 
                                      Sd_star, Sd_star_err, l_star_kpc, aCO=alpha_co10_sl24, R21=R_21)
    
    # add to pystructure table
    struct['INT_VAL_P_DE_SL24' ] = P_DE_sl24
    struct['INT_UC_P_DE_SL24'] = P_DE_err_sl24


    ######## constant alpha_CO ########
    
    # compute molecular gas surface density
    Sd_mol_mw = co10 * alpha_co10_mw
    Sd_mol_err_mw = co10_err * alpha_co10_mw
    struct['INT_VAL_SD_MOL_MW'] = Sd_mol_mw * np.cos(i)
    struct['INT_UC_SD_MOL_MW'] = Sd_mol_err_mw * np.cos(i)

    # compute kpc-scale ISM equilibrium pressure
    P_DE_mw, P_DE_err_mw = get_ism_pressure(co21*np.cos(i), co21_err*np.cos(i), hi*np.cos(i), hi_err*np.cos(i), 
                                      Sd_star, Sd_star_err, l_star_kpc, aCO=alpha_co10_mw, R21=R_21)
    
    # add to pystructure table
    struct['INT_VAL_P_DE_MW' ] = P_DE_mw
    struct['INT_UC_P_DE_MW'] = P_DE_err_mw

    
    ######## save new pystructure file ########
    fpath_new = pystruct_file
    np.save(fpath_new, struct)

# EMPIRE

In [5]:
for glxy, res, in zip(empire_galaxies, empire_res_list):

    # pystructure file
    pystruct_dir = data_dir + 'EMPIRE/pystruct/'
    pystruct_file = pystruct_dir + '%s_data_struct_%s_spb2.npy' % (glxy, res)
    struct = np.load(pystruct_file, allow_pickle = True).item()
    
    ######## get data from structure ########

    # load distance and inclination
    dist_Mpc = struct['dist_mpc']  # distance to the galaxy in Mpc
    incl = struct['incl_deg']  # inclination of the galaxy in degree
    i = np.deg2rad(incl)  # convert from deg to rad
    
    # # conversion angular to linear scale
    dist_pc = dist_Mpc * 1e6
    kpc2as = 180/np.pi * 3600 / dist_pc * 1e3
    as2kpc = 1/kpc2as
    
    # CO(1-0) from EMPIRE/PAWS
    co10 = np.array(struct['INT_VAL_CO10'])
    co10_err = np.array(struct['INT_UC_CO10'])
    
    # HCN(1-0) from ALMOND
    hcn = np.array(struct['INT_VAL_HCN10'])
    hcn_err = np.array(struct['INT_UC_HCN10'])
    
    # SFR surface density from z0MGS (GALEX+WISE)
    Sd_sfr = np.array(struct['INT_VAL_SFR_Z0MGS'])
    Sd_sfr_err = np.array(struct['INT_UC_SFR_Z0MGS'])
    
    # HI 21cm line intensity, K km/s (several surveys)
    hi = np.array(struct['INT_VAL_HI_21CM'])
    hi_err = np.array(struct['INT_UC_HI_21CM'])
    
    # Stellar mass surface density, Msun/pc2 (Spitzer 3.6um; Querejeta+15)
    Sd_star = np.array(struct['INT_VAL_MSTAR']) * np.cos(i)
    Sd_star_err = np.array(struct['INT_UC_MSTAR']) * np.cos(i)
    struct['INT_VAL_SD_STAR'] = Sd_star
    struct['INT_UC_SD_STAR'] = Sd_star_err
       
    # get CO(2-1)/CO(1-0) line ratio (SFR calibration from Schinnerer & Leroy 24)
    R_21 = struct['INT_VAL_R21_SL24'] 

    # get conversion factors
    alpha_co21_sl24 = struct['INT_VAL_ALPHACO21_SL24']
    alpha_co10_sl24 = struct['INT_VAL_ALPHACO10_SL24']

    
    ######## compute physical quantities ########

    # compute atomic gas surface density
    Sd_atom, Sd_atom_err = get_atomic_gas_surface_density(W_21cm=hi, W_21cm_err=hi_err)
    struct['INT_VAL_SD_ATOM'] = Sd_atom * np.cos(i)
    struct['INT_UC_SD_ATOM'] = Sd_atom_err * np.cos(i)

    # convert to dense gas surface density
    Sd_dense, Sd_dense_err = get_dense_gas_surface_density(W_hcn=hcn, W_hcn_err=hcn_err, aHCN=alpha_hcn)
    struct['INT_VAL_SD_DENSE'] = Sd_dense * np.cos(i)
    struct['INT_UC_SD_DENSE'] = Sd_dense_err * np.cos(i)

    # compute CO(2-1) intensity from CO(1-0) using above line ratio
    co21 = co10 * R_21
    co21_err = co10_err * R_21
    struct['INT_VAL_CO21'] = co21
    struct['INT_UC_CO21'] = co21_err

    # CO(1-0) intensities for stacking
    R_21_cube = np.broadcast_to(R_21, np.shape(struct['SPEC_VAL_CO10'].T)).T  # broadcast to cube shape
    struct['SPEC_VAL_CO21'] = np.copy(struct['SPEC_VAL_CO10']) * R_21_cube
    R_21_cube_shuff = np.broadcast_to(R_21, np.shape(struct['SPEC_VAL_SHUFFCO10'].T)).T  # broadcast to cube shape
    struct['SPEC_VAL_SHUFFCO21'] = np.copy(struct['SPEC_VAL_SHUFFCO10']) * R_21_cube_shuff
    struct['SPEC_VCHAN0_SHUFFCO21'] = np.copy(struct['SPEC_VCHAN0_SHUFFCO10'])
    struct['SPEC-DELTAV_SHUFFCO21'] = np.copy(struct['SPEC-DELTAV_SHUFFCO10'])

    
    ######## varying alpha_CO ########
    
    # compute molecular gas surface density
    Sd_mol_sl24 = co10 * alpha_co10_sl24
    Sd_mol_err_sl24 = co10_err * alpha_co10_sl24
    struct['INT_VAL_SD_MOL_SL24'] = Sd_mol_sl24 * np.cos(i)
    struct['INT_UC_SD_MOL_SL24'] = Sd_mol_err_sl24 * np.cos(i)

    # get disc scale length
    l_star_as = float(phangs_sample_table['size_scalelength'][phangs_sample_table['name']==glxy].iloc[0])
    l_star_kpc = l_star_as * as2kpc  # [kpc]
    
    # compute kpc-scale ISM equilibrium pressure
    P_DE_sl24, P_DE_err_sl24 = get_ism_pressure(co21*np.cos(i), co21_err*np.cos(i), hi*np.cos(i), hi_err*np.cos(i), 
                                                Sd_star, Sd_star_err, l_star_kpc, aCO=alpha_co10_sl24, R21=R_21)
    
    # add to pystructure table
    struct['INT_VAL_P_DE_SL24' ] = P_DE_sl24
    struct['INT_UC_P_DE_SL24'] = P_DE_err_sl24


    ######## constant alpha_CO ########
    
    # compute molecular gas surface density
    Sd_mol_mw = co10 * alpha_co10_mw
    Sd_mol_err_mw = co10_err * alpha_co10_mw
    struct['INT_VAL_SD_MOL_MW'] = Sd_mol_mw * np.cos(i)
    struct['INT_UC_SD_MOL_MW'] = Sd_mol_err_mw * np.cos(i)

    # compute ISM equilibrium pressure
    P_DE_mw, P_DE_err_mw = get_ism_pressure(co21*np.cos(i), co21_err*np.cos(i), hi*np.cos(i), hi_err*np.cos(i), 
                                            Sd_star, Sd_star_err, l_star_kpc, aCO=alpha_co10_mw, R21=R_21)
    
    # add to pystructure table
    struct['INT_VAL_P_DE_MW' ] = P_DE_mw
    struct['INT_UC_P_DE_MW'] = P_DE_err_mw

    
    ######## save new pystructure file ########
    fpath_new = pystruct_file
    np.save(fpath_new, struct)